In [ ]:
import sys

sys.path.append("../../")
import time
import torch
from losses.basic_losses import CategoricalCrossentropyLoss, KLDivergenceLoss
from agents.random import RandomAgent
from agents.muzero import MuZeroAgent
from agent_configs.muzero_config import MuZeroConfig
from game_configs.tictactoe_config import TicTacToeConfig
from agents.tictactoe_expert import TicTacToeBestAgent
from modules.world_models.muzero_world_model import MuzeroWorldModel

# Ensure we use CPU for fairness/comparibility or GPU if available
device = "cpu"  # or "cuda" if available
print(f"Using device: {device}")

# MuZero Benchmark (Iterative vs Batched)

In [ ]:
params = {
    "num_simulations": 25,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "n_step": 10,
    "root_dirichlet_alpha": 0.25,
    "residual_layers": [(24, 3, 1)],
    "reward_dense_layer_widths": [],
    "reward_conv_layers": [(16, 1, 1)],
    "actor_dense_layer_widths": [],
    "actor_conv_layers": [(16, 1, 1)],
    "critic_dense_layer_widths": [],
    "critic_conv_layers": [(16, 1, 1)],
    "to_play_dense_layer_widths": [],
    "to_play_conv_layers": [(16, 1, 1)],
    "known_bounds": [-1, 1],
    "support_range": None,
    "minibatch_size": 8,
    "replay_buffer_size": 100000,
    "gumbel": False,
    "gumbel_m": 16,
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "training_steps": 20000,  # Reduced for benchmark speed
    "transfer_interval": 1,
    "num_workers": 4,
    "world_model_cls": MuzeroWorldModel,
    "search_batch_size": 0,  # Iterative
    "use_virtual_mean": False,
    "virtual_loss": 3.0,
    "use_torch_compile": True,
    "use_mixed_precision": True,
    "use_quantization": False,
}

game_config = TicTacToeConfig()

In [ ]:
print("--- Running MuZero Batched Search Max Fast ---")
params_batched = params.copy()
params_batched["num_workers"] = 4
params_batched["search_batch_size"] = 5
params_batched["use_virtual_mean"] = True
params_batched["use_mixed_precision"] = True
params_batched["use_torch_compile"] = True
params_batched["use_quantization"] = True
params_batched["qat"] = True
params_batched["transfer_interval"] = 100

# params_batched["num_envs_per_worker"] = 4

env_batch = TicTacToeConfig().make_env()
config_batch = MuZeroConfig(config_dict=params_batched, game_config=game_config)

agent_batch = MuZeroAgent(
    env=env_batch,
    config=config_batch,
    name="muzero_batched_bench_fast",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_batch.checkpoint_interval = 100
agent_batch.test_interval = 1000
agent_batch.test_trials = 100

start_time = time.time()
agent_batch.train()
end_time = time.time()
print(f"MuZero Batched Time: {end_time - start_time:.2f}s")

(raylet) [2026-01-28 14:35:13,025 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 16.503 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.


Started recording episode 5399 to checkpoints/muzero_batched_bench_fast/step_18000/videos/muzero_batched_bench_fast/episode_005399.mp4
Stopped recording episode 5399. Recorded 8 frames.
average score: 0.4
Test score {'score': 0.4, 'max_score': 1, 'min_score': 0}
plotting score
plotting policy_loss
plotting value_loss
plotting reward_loss
plotting to_play_loss
plotting cons_loss
plotting loss
plotting test_score
  subkey score
  subkey max_score
  subkey min_score
plotting episode_length
plotting root_children_values
plotting test_score_vs_random
  subkey score
  subkey player_1_score
  subkey player_2_score
  subkey player_1_win%
  subkey player_2_win%
plotting test_score_vs_tictactoe_expert
  subkey score
  subkey player_1_score
  subkey player_2_score
  subkey player_1_win%
  subkey player_2_win%
plotting q_loss
plotting sigma_loss
plotting vqvae_commitment_cost


(raylet) [2026-01-28 14:35:23,119 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 16.5 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.


plotting policy_entropy
plotting value_diff
plotting policy_improvement
  subkey network
  subkey search
plotting latent viz latent_root using umap
  Saving latent viz to checkpoints/muzero_batched_bench_fast/graphs/muzero_batched_bench_fast_latent_root_umap.png
18000
actions shape torch.Size([8, 5])
target value shape torch.Size([8, 6])
predicted values shape torch.Size([8, 6, 1])
target rewards shape torch.Size([8, 6])
predicted rewards shape torch.Size([8, 6, 1])
target to plays shape torch.Size([8, 6, 2])
predicted to_plays shape torch.Size([8, 6, 2])
masks shape torch.Size([8, 6]) torch.Size([8, 6])
actions tensor([[4, 0, 1, 7, 3],
        [5, 6, 2, 8, 1],
        [5, 3, 0, 8, 2],
        [7, 2, 0, 4, 4],
        [7, 3, 0, 4, 4],
        [2, 0, 8, 3, 5],
        [4, 5, 2, 7, 6],
        [4, 0, 4, 4, 4]])
target value tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9606, -0.9703,  0.9801, -0.9900,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0

(raylet) [2026-01-28 14:35:33,202 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 16.4999 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.
(raylet) [2026-01-28 14:35:43,279 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 15.6863 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.
(raylet) [2026-01-28 14:35:53,337 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 15.5152 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.


...

Player 0 (player_1) win percentage vs random: 98.0 and average score: 0.98
Testing Player 1 vs Agent random
Player 0 random action: 4
Player 1 prediction: (tensor([0.2800, 0.0400, 0.0800, 0.0400, 0.0000, 0.1200, 0.2800, 0.0400, 0.1200]), tensor([0.2800, 0.0400, 0.0800, 0.0400, 0.0000, 0.1200, 0.2800, 0.0400, 0.1200]), -0.27205003446030945, tensor(0), {'network_policy': tensor([0.2331, 0.0712, 0.1057, 0.0667, 0.0000, 0.0667, 0.2331, 0.0667, 0.1570]), 'network_value': -0.26199525594711304, 'search_policy': tensor([0.2800, 0.0400, 0.0800, 0.0400, 0.0000, 0.1200, 0.2800, 0.0400, 0.1200]), 'search_value': -0.27205003446030945, 'root_children_values': tensor([0.2329, 0.6119, 0.2703, 0.5229, 0.0000, 0.4629, 0.1628, 0.4844, 0.2776])})
action: 0
Player 0 random action: 8
Player 1 prediction: (tensor([0.0000, 0.0800, 0.2000, 0.3600, 0.0000, 0.0400, 0.2800, 0.0400, 0.0000]), tensor([0.0000, 0.0800, 0.2000, 0.3600, 0.0000, 0.0400, 0.2800, 0.0400, 0.0000]), -0.12228531138334361, tensor(3), {'networ

*** SIGTERM received at time=1769630451 ***
PC: @        0x19d7dd3cc  (unknown)  __psynch_cvwait
    @        0x130f01f4c  (unknown)  absl::lts_20230802::AbslFailureSignalHandler()
    @        0x19d8556a4  (unknown)  _sigtramp
    @        0x19d81c0e0  (unknown)  _pthread_cond_wait
    @        0x10c958944  (unknown)  __kmp_suspend_64<>()
    @        0x10c93e454  (unknown)  kmp_flag_64<>::wait()
    @        0x10c93c390  (unknown)  __kmp_hyper_barrier_gather()
    @        0x10c93ba90  (unknown)  __kmp_join_barrier()
    @        0x10c92008c  (unknown)  __kmp_internal_join
    @        0x10c91f8ac  (unknown)  __kmp_join_call
    @        0x10c9130d8  (unknown)  __kmpc_fork_call
    @        0x1735c5c7c  (unknown)  at::native::(anonymous namespace)::compute_columns2d()
    @        0x1735c2470  (unknown)  at::native::slow_conv2d_forward_out_cpu()
    @        0x1735cd87c  (unknown)  at::native::slow_conv2d_forward_cpu()
    @        0x173da59f4  (unknown)  at::_ops::_slow_conv2d_forwa

SystemExit: 15

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
(raylet) [2026-01-28 15:00:52,350 E 12900 162317] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2026-01-28_10-52-52_452733_12886 is over 95% full, available space: 15.9491 GB; capacity: 460.432 GB. Object creation will fail if spilling is required.


: 

In [ ]:
print("--- Running MuZero Iterative Search (Batch=0) ---")
env_iter = TicTacToeConfig().make_env()
config_iter = MuZeroConfig(config_dict=params, game_config=game_config)
config_iter.search_batch_size = 0  # Explicitly set

agent_iter = MuZeroAgent(
    env=env_iter,
    config=config_iter,
    name="muzero_iterative_bench",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_iter.checkpoint_interval = 100
agent_iter.test_interval = 1000
agent_iter.test_trials = 100

start_time = time.time()
agent_iter.train()
end_time = time.time()
print(f"MuZero Iterative Time: {end_time - start_time:.2f}s")

In [ ]:
print("--- Running MuZero Iterative Search (Batch=1) ---")
env_iter = TicTacToeConfig().make_env()
config_iter = MuZeroConfig(config_dict=params, game_config=game_config)
config_iter.search_batch_size = 1  # Explicitly set

agent_iter = MuZeroAgent(
    env=env_iter,
    config=config_iter,
    name="muzero_iterative_bench",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_iter.checkpoint_interval = 100
agent_iter.test_interval = 1000
agent_iter.test_trials = 100

start_time = time.time()
agent_iter.train()
end_time = time.time()
print(f"MuZero Iterative Time: {end_time - start_time:.2f}s")

In [ ]:
print("--- Running MuZero Batched Search (Batch=5) ---")
params_batched = params.copy()
params_batched["search_batch_size"] = 5

env_batch = TicTacToeConfig().make_env()
config_batch = MuZeroConfig(config_dict=params_batched, game_config=game_config)
config_batch.search_batch_size = 5  # Explicitly set

agent_batch = MuZeroAgent(
    env=env_batch,
    config=config_batch,
    name="muzero_batched_bench_size_5",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_batch.checkpoint_interval = 100
agent_batch.test_interval = 1000
agent_batch.test_trials = 100

start_time = time.time()
agent_batch.train()
end_time = time.time()
print(f"MuZero Batched Time: {end_time - start_time:.2f}s")

In [ ]:
print("--- Running MuZero Batched Search (Batch=5) ---")
params_batched = params.copy()
params_batched["search_batch_size"] = 5

env_batch = TicTacToeConfig().make_env()
config_batch = MuZeroConfig(config_dict=params_batched, game_config=game_config)
config_batch.search_batch_size = 5  # Explicitly set

agent_batch = MuZeroAgent(
    env=env_batch,
    config=config_batch,
    name="muzero_batched_bench_size_5",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_batch.checkpoint_interval = 100
agent_batch.test_interval = 1000
agent_batch.test_trials = 100

start_time = time.time()
agent_batch.train()
end_time = time.time()
print(f"MuZero Batched Time: {end_time - start_time:.2f}s")

In [ ]:
print("--- Running MuZero Batched Search (Batch=5) Virtual Mean ---")
params_batched = params.copy()
params_batched["search_batch_size"] = 5
params_batched["use_virtual_mean"] = True

env_batch = TicTacToeConfig().make_env()
config_batch = MuZeroConfig(config_dict=params_batched, game_config=game_config)
config_batch.search_batch_size = 5  # Explicitly set

agent_batch = MuZeroAgent(
    env=env_batch,
    config=config_batch,
    name="muzero_batched_bench_size_5_virtual_mean_1",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_batch.checkpoint_interval = 100
agent_batch.test_interval = 1000
agent_batch.test_trials = 100

start_time = time.time()
agent_batch.train()
end_time = time.time()
print(f"MuZero Batched Time: {end_time - start_time:.2f}s")

# Gumbel MuZero Benchmark (Iterative vs Batched)

In [ ]:
params = {
    "num_simulations": 25,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "n_step": 10,
    "root_dirichlet_alpha": 0.25,
    "residual_layers": [(24, 3, 1)],
    "reward_dense_layer_widths": [],
    "reward_conv_layers": [(16, 1, 1)],
    "actor_dense_layer_widths": [],
    "actor_conv_layers": [(16, 1, 1)],
    "critic_dense_layer_widths": [],
    "critic_conv_layers": [(16, 1, 1)],
    "to_play_dense_layer_widths": [],
    "to_play_conv_layers": [(16, 1, 1)],
    "known_bounds": [-1, 1],
    "support_range": None,
    "minibatch_size": 8,
    "replay_buffer_size": 100000,
    "gumbel": True,
    "gumbel_m": 4,
    "policy_loss_function": KLDivergenceLoss(),
    "training_steps": 20000,  # Reduced for benchmark speed
    "transfer_interval": 1,
    "num_workers": 4,
    "world_model_cls": MuzeroWorldModel,
    "search_batch_size": 0,  # Iterative
    "use_virtual_mean": False,
    "virtual_loss": 3.0,
}

game_config = TicTacToeConfig()

params_gumbel = params.copy()

In [ ]:
print("--- Running Gumbel MuZero Iterative Search (Batch=1) ---")
params_gumbel["search_batch_size"] = 0

env_gumbel = TicTacToeConfig().make_env()
config_gumbel = MuZeroConfig(config_dict=params_gumbel, game_config=game_config)

agent_gumbel = MuZeroAgent(
    env=env_gumbel,
    config=config_gumbel,
    name="gumbel_iterative_bench",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_gumbel.checkpoint_interval = 100
agent_gumbel.test_interval = 1000
agent_gumbel.test_trials = 100

start_time = time.time()
agent_gumbel.train()
end_time = time.time()
print(f"Gumbel Iterative Time: {end_time - start_time:.2f}s")

In [ ]:
print("--- Running Gumbel MuZero Batched Search (Batch=5) ---")
params_gumbel_batch = params_gumbel.copy()
params_gumbel_batch["search_batch_size"] = 5
params_gumbel_batch["use_virtual_mean"] = True

env_gumbel_batch = TicTacToeConfig().make_env()
config_gumbel_batch = MuZeroConfig(
    config_dict=params_gumbel_batch, game_config=game_config
)

agent_gumbel_batch = MuZeroAgent(
    env=env_gumbel_batch,
    config=config_gumbel_batch,
    name="gumbel_batched_bench",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)
agent_gumbel_batch.checkpoint_interval = 100
agent_gumbel_batch.test_interval = 1000
agent_gumbel_batch.test_trials = 100

start_time = time.time()
agent_gumbel_batch.train()
end_time = time.time()
print(f"Gumbel Batched Time: {end_time - start_time:.2f}s")